In [1]:
import json,httplib,urllib

In [2]:
connection = httplib.HTTPSConnection('api.parse.com', 443)
params = urllib.urlencode({"where":json.dumps({
       "createdAt": {
             #"$gte": '2015-09-21T05:16:11.880Z'
             "$gte": '2015-11-01T05:16:11.880Z'
       }
     })})
connection.connect()
connection.request('GET', '/1/classes/Expenses?%s' % params, '', {
       "X-Parse-Application-Id": "dCxojn2rtpic9RlfY6mjwTqWHNjHqsUOvwzPTl0Y",
       "X-Parse-Master-Key": "Cu0iTrTWgntiqx4hbKesI25HuuBZKhIaxdKiJyWM"
     })
result = json.loads(connection.getresponse().read())
#print result
result = result['results']
#print result

In [3]:
from collections import namedtuple
def convert(dictionary):
    return namedtuple('GenericDict', dictionary.keys())(**dictionary)
    #_tuple = namedtuple('GenericDict', dictionary.keys())(**dictionary)
    #return namedtuple('ExpenseParseData', _tuple._fields+('TanggalDatetime','createdAtDateTime',))
tuple_expense_data = namedtuple('ExpenseParseData', ['username', 'Description', 'objectId', 'Cash', 'Value', 'ACL', 'Saldo', 'Tanggal', 'updatedAt', 'Type', 'createdAt', 'tanggal_datetime', 'createdAt_datetime'])

In [4]:
import datetime
import pytz
import dateutil.parser

tz =  pytz.timezone('Asia/Singapore')

for item in result:
    #_when = dateutil.parser.parse(item.Tanggal)
    #item.Tanggal = _when.astimezone(tz)
    item['tanggal_datetime'] = dateutil.parser.parse(item['Tanggal']['iso']).astimezone(tz)
    item['createdAt_datetime'] = dateutil.parser.parse(item['createdAt']).astimezone(tz)

In [5]:
#_result = [convert(item) for item in result]
_result = [tuple_expense_data(**item) for item in result]
result = _result
#print result

In [6]:
import openpyxl
wb = openpyxl.load_workbook('test_py_styles.xlsx')
wb_data = openpyxl.load_workbook('test_py_styles.xlsx', data_only = True)
#sheet = wb.get_sheet_by_name('Sheet')
#sheet_data = wb_data.get_sheet_by_name('Sheet')
#sheet = wb.get_sheet_by_name('MingguanParseFeiSept')
#sheet_data = wb_data.get_sheet_by_name('MingguanParseFeiSept')
#sheet = wb.get_sheet_by_name('MingguanParseBillySept')
#sheet_data = wb_data.get_sheet_by_name('MingguanParseBillySept')

In [13]:
sheet = wb.create_sheet()
sheet.title = "MingguanParseBillyOct"
#sheet.title = "MingguanParseFeiOct"

In [14]:
titles = ['Date','InputDate','Keterangan','CashYa?','Value','Saldo','Category','makan','dining','groceries','transport','comm','giving','other','recurr','atm','indo','big','saving','correction_plus','correction_minus']
column_index = 1
for string in titles:
    sheet.cell(column=column_index,row=1).value = string
    column_index = column_index + 1

In [15]:
titles_column = dict(zip(titles,range(1,22)))
titles_column

{'CashYa?': 4,
 'Category': 7,
 'Date': 1,
 'InputDate': 2,
 'Keterangan': 3,
 'Saldo': 6,
 'Value': 5,
 'atm': 16,
 'big': 18,
 'comm': 12,
 'correction_minus': 21,
 'correction_plus': 20,
 'dining': 9,
 'giving': 13,
 'groceries': 10,
 'indo': 17,
 'makan': 8,
 'other': 14,
 'recurr': 15,
 'saving': 19,
 'transport': 11}

In [16]:
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row_index = 2
for item in result:
    #if item.username == 'feifei' and item.tanggal_datetime.day>1 and item.tanggal_datetime.day<9 and item.tanggal_datetime.month==11:
    if item.username == 'billy' and item.tanggal_datetime.day>1 and item.tanggal_datetime.day<9 and item.tanggal_datetime.month==11:
    #if item.username == 'billy' and ((item.tanggal_datetime.day>25 and item.tanggal_datetime.month==10) or ((item.tanggal_datetime.day<2 and item.tanggal_datetime.month==11))):
    #if item.username == 'feifei' and ((item.tanggal_datetime.day>25 and item.tanggal_datetime.month==10) or ((item.tanggal_datetime.day<2 and item.tanggal_datetime.month==11))):
        if row_index == 2:
            for i in range (8,22): 
                sheet.cell(column=i,row=row_index).value = 0
            if item.Cash:
                sheet.cell(column=titles_column[item.Type],row=row_index).value = item.Value
        else:
            for i in range (8,22): 
                sheet.cell(column=i,row=row_index).value = sheet.cell(column=i,row=row_index-1).value
            if item.Cash:
                sheet.cell(column=titles_column[item.Type],row=row_index).value = item.Value+\
                  sheet.cell(column=titles_column[item.Type],row=row_index-1).value
        
        sheet.cell(column=titles_column['Keterangan'],row=row_index).value = item.Description
        sheet.cell(column=titles_column['CashYa?'],row=row_index).value = item.Cash
        sheet.cell(column=titles_column['Value'],row=row_index).value = item.Value
        sheet.cell(column=titles_column['Saldo'],row=row_index).value = item.Saldo
        sheet.cell(column=titles_column['Category'],row=row_index).value = item.Type
        
        sheet[get_column_letter(titles_column['Date'])+str(row_index)] = item.tanggal_datetime
        sheet[get_column_letter(titles_column['Date'])+str(row_index)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
        sheet[get_column_letter(titles_column['InputDate'])+str(row_index)] = item.createdAt_datetime
        sheet[get_column_letter(titles_column['InputDate'])+str(row_index)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
        row_index = row_index + 1

In [17]:
sheet.column_dimensions['A'].width = 32
sheet.column_dimensions['B'].width = 32
sheet.column_dimensions['C'].width = 25

In [18]:
wb.save('test_py_styles.xlsx')

In [17]:
print result

[ExpenseParseData(username=u'billy', Description=u'Rekap makan', objectId=u'Le0M1WO5hz', Cash=True, Value=10, ACL={u'Glz7N1JoPa': {u'read': True, u'write': True}}, Saldo=37, Tanggal={u'iso': u'2015-10-25T16:00:00.000Z', u'__type': u'Date'}, updatedAt=u'2015-10-27T01:14:20.303Z', Type=u'makan', createdAt=u'2015-10-27T01:14:20.303Z', tanggal_datetime=datetime.datetime(2015, 10, 26, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), createdAt_datetime=datetime.datetime(2015, 10, 27, 9, 14, 20, 303000, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>)), ExpenseParseData(username=u'feifei', Description=u'Taxi', objectId=u'Z0SwQQttY7', Cash=True, Value=20, ACL={u'XASf98BKrJ': {u'read': True, u'write': True}}, Saldo=77, Tanggal={u'iso': u'2015-10-24T00:00:00.000Z', u'__type': u'Date'}, updatedAt=u'2015-10-27T02:03:44.755Z', Type=u'transport', createdAt=u'2015-10-27T02:03:44.755Z', tanggal_datetime=datetime.datetime(2015, 10, 24, 8, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00

### TESTING

In [2]:
connection = httplib.HTTPSConnection('api.parse.com', 443)
params = urllib.urlencode({"order":"-createdAt","limit":10})
connection.connect()
connection.request('GET', '/1/classes/Expenses?%s' % params, '', {
       "X-Parse-Application-Id": "dCxojn2rtpic9RlfY6mjwTqWHNjHqsUOvwzPTl0Y",
       "X-Parse-REST-API-Key": "XzDkpbFjOPkJhW9fBHCPZGZ3mGgwk1C0GHFVeMKb"
     })
result = json.loads(connection.getresponse().read())
print result

{u'results': [{u'username': u'billy', u'Description': u'test transport', u'objectId': u'4bhqOK48l3', u'Value': 8, u'Saldo': -20, u'Tanggal': {u'iso': u'2015-09-15T14:08:24.176Z', u'__type': u'Date'}, u'updatedAt': u'2015-09-15T14:08:34.174Z', u'Type': u'transport', u'createdAt': u'2015-09-15T14:08:34.174Z'}, {u'username': u'billy', u'Description': u'test lagih', u'objectId': u'M3BzlUC3j4', u'Value': 12, u'Saldo': -12, u'Tanggal': {u'iso': u'2015-09-05T00:00:00.000Z', u'__type': u'Date'}, u'updatedAt': u'2015-09-15T14:07:03.588Z', u'Type': u'correction_minus', u'createdAt': u'2015-09-15T14:07:03.588Z'}, {u'username': u'billy', u'Description': u'test', u'objectId': u'axiiqDr4Qh', u'Value': 7, u'Saldo': -12, u'Tanggal': {u'iso': u'2015-09-15T14:04:43.316Z', u'__type': u'Date'}, u'updatedAt': u'2015-09-15T14:05:13.379Z', u'Type': u'correction_minus', u'createdAt': u'2015-09-15T14:05:13.379Z'}, {u'username': u'feifei', u'Description': u'buah', u'objectId': u'0fA8YLVPvu', u'Value': 11, u'Sal

In [2]:
connection = httplib.HTTPSConnection('api.parse.com', 443)
params = urllib.urlencode({"order":"-createdAt","limit":10})
connection.connect()
connection.request('GET', '/1/classes/Expenses?%s' % params, '', {
       "X-Parse-Application-Id": "dCxojn2rtpic9RlfY6mjwTqWHNjHqsUOvwzPTl0Y",
       "X-Parse-Master-Key": "Cu0iTrTWgntiqx4hbKesI25HuuBZKhIaxdKiJyWM"
     })
result = json.loads(connection.getresponse().read())
print result

{u'results': [{u'username': u'billy', u'Description': u'Ezlink top up', u'objectId': u'mHm3cKTvdw', u'Cash': False, u'Value': 30, u'ACL': {u'Glz7N1JoPa': {u'read': True, u'write': True}}, u'Saldo': 39, u'Tanggal': {u'iso': u'2015-09-25T02:54:35.614Z', u'__type': u'Date'}, u'updatedAt': u'2015-09-25T02:55:00.784Z', u'Type': u'transport', u'createdAt': u'2015-09-25T02:55:00.784Z'}, {u'username': u'billy', u'Description': u'Internet starhub kos', u'objectId': u'NxrUDIrQyh', u'Cash': False, u'Value': 29.9, u'ACL': {u'Glz7N1JoPa': {u'read': True, u'write': True}}, u'Saldo': 39, u'Tanggal': {u'iso': u'2015-09-24T08:28:47.549Z', u'__type': u'Date'}, u'updatedAt': u'2015-09-24T08:29:52.186Z', u'Type': u'comm', u'createdAt': u'2015-09-24T08:29:52.186Z'}, {u'username': u'billy', u'Description': u'Makan waktu libur', u'objectId': u'BUgh9Bt0RQ', u'Cash': True, u'Value': 5, u'ACL': {u'Glz7N1JoPa': {u'read': True, u'write': True}}, u'Saldo': 39, u'Tanggal': {u'iso': u'2015-09-24T08:24:08.170Z', u'__

In [4]:
connection = httplib.HTTPSConnection('api.parse.com', 443)
params = urllib.urlencode({"where":json.dumps({
       "objectId": "B9K1pfNKM8"
     })})
connection.connect()
connection.request('GET', '/1/classes/Expenses?%s' % params, '', {
       "X-Parse-Application-Id": "dCxojn2rtpic9RlfY6mjwTqWHNjHqsUOvwzPTl0Y",
       "X-Parse-Master-Key": "Cu0iTrTWgntiqx4hbKesI25HuuBZKhIaxdKiJyWM"
     })
result = json.loads(connection.getresponse().read())
print result

{u'results': [{u'username': u'feifei', u'Description': u'Kaya toast for brekfast', u'objectId': u'B9K1pfNKM8', u'Cash': True, u'Value': 5, u'ACL': {u'XASf98BKrJ': {u'read': True, u'write': True}}, u'Saldo': 30, u'Tanggal': {u'iso': u'2015-09-21T05:25:42.062Z', u'__type': u'Date'}, u'updatedAt': u'2015-09-21T05:16:11.880Z', u'Type': u'makan', u'createdAt': u'2015-09-21T05:16:11.880Z'}]}


In [19]:
connection = httplib.HTTPSConnection('api.parse.com', 443)
params = urllib.urlencode({"where":json.dumps({
       "createdAt": {
             #"$gte": '2015-09-21T05:16:11.880Z'
             "$gte": '2015-10-03T05:16:11.880Z'
       }
     })})
connection.connect()
connection.request('GET', '/1/classes/Expenses?%s' % params, '', {
       "X-Parse-Application-Id": "dCxojn2rtpic9RlfY6mjwTqWHNjHqsUOvwzPTl0Y",
       "X-Parse-Master-Key": "Cu0iTrTWgntiqx4hbKesI25HuuBZKhIaxdKiJyWM"
     })
result = json.loads(connection.getresponse().read())
#print result
result = result['results']
#print result

In [43]:
_keys = []
for key in result[0]:
    _keys.append(key)
str(_keys).replace('u','')

"['sername', 'Description', 'objectId', 'Cash', 'Vale', 'ACL', 'Saldo', 'Tanggal', 'pdatedAt', 'Type', 'createdAt']"

In [3]:
from collections import namedtuple
def convert(dictionary):
    return namedtuple('GenericDict', dictionary.keys())(**dictionary)
    #_tuple = namedtuple('GenericDict', dictionary.keys())(**dictionary)
    #return namedtuple('ExpenseParseData', _tuple._fields+('TanggalDatetime','createdAtDateTime',))
tuple_expense_data = namedtuple('ExpenseParseData', ['username', 'Description', 'objectId', 'Cash', 'Value', 'ACL', 'Saldo', 'Tanggal', 'updatedAt', 'Type', 'createdAt', 'tanggal_datetime', 'createdAt_datetime'])

In [63]:
result[0]

{u'ACL': {u'XASf98BKrJ': {u'read': True, u'write': True}},
 u'Cash': True,
 u'Description': u'Kaya toast for brekfast',
 u'Saldo': 30,
 u'Tanggal': {u'__type': u'Date', u'iso': u'2015-09-21T05:25:42.062Z'},
 u'Type': u'makan',
 u'Value': 5,
 '_createdAt_datetime': datetime.datetime(2015, 9, 21, 13, 16, 11, 880000, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>),
 '_tanggal_datetime': datetime.datetime(2015, 9, 21, 13, 25, 42, 62000, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>),
 u'createdAt': u'2015-09-21T05:16:11.880Z',
 u'objectId': u'B9K1pfNKM8',
 u'updatedAt': u'2015-09-21T05:16:11.880Z',
 u'username': u'feifei'}

In [59]:
tuple_expense_data(**(result[0]).append( 2*[0]) )

AttributeError: 'dict' object has no attribute 'append'

In [21]:
import datetime
import pytz
import dateutil.parser

tz =  pytz.timezone('Asia/Singapore')

for item in result:
    #_when = dateutil.parser.parse(item.Tanggal)
    #item.Tanggal = _when.astimezone(tz)
    item['tanggal_datetime'] = dateutil.parser.parse(item['Tanggal']['iso']).astimezone(tz)
    item['createdAt_datetime'] = dateutil.parser.parse(item['createdAt']).astimezone(tz)
    

In [22]:
#_result = [convert(item) for item in result]
_result = [tuple_expense_data(**item) for item in result]
result = _result
#print result

In [25]:
type(item.Type)

unicode

In [11]:
for item in result:
    #print item.username, item.Description, item.Cash, item.Value, item.Saldo, item.tanggal_datetime, item.Type, item.createdAt_datetime
    print item.tanggal_datetime, item.createdAt_datetime, item.username, item.Description, item.Cash, item.Value, item.Saldo, item.Type

AttributeError: 'dict' object has no attribute 'tanggal_datetime'

In [5]:
import openpyxl
wb = openpyxl.load_workbook('test_py_styles.xlsx')
wb_data = openpyxl.load_workbook('test_py_styles.xlsx', data_only = True)
#sheet = wb.get_sheet_by_name('Sheet')
#sheet_data = wb_data.get_sheet_by_name('Sheet')
#sheet = wb.get_sheet_by_name('MingguanParseFeiSept')
#sheet_data = wb_data.get_sheet_by_name('MingguanParseFeiSept')
sheet = wb.get_sheet_by_name('MingguanParseBillySept')
sheet_data = wb_data.get_sheet_by_name('MingguanParseBillySept')


In [63]:
new_work_sheet = wb.create_sheet()
#new_work_sheet.title = "MingguanParseFeiSept"
new_work_sheet.title = "MingguanParseBillySept"

In [55]:
wb.remove_sheet(new_work_sheet)

In [30]:
wb.save('test_py_styles.xlsx')

In [7]:
new_work_sheet = sheet

In [8]:
#titles = ['Date','InputDate','User','Keterangan','CashYa?','Value','Saldo','Category']
titles = ['Date','InputDate','Keterangan','CashYa?','Value','Saldo','Category','makan','dining','groceries','transport','comm','giving','other','recurr','atm','indo','big','saving','correction_plus','correction_minus']
column_index = 1
for string in titles:
    new_work_sheet.cell(column=column_index,row=37).value = string
    column_index = column_index + 1

In [9]:
titles_column = dict(zip(titles,range(1,22)))
titles_column

{'CashYa?': 4,
 'Category': 7,
 'Date': 1,
 'InputDate': 2,
 'Keterangan': 3,
 'Saldo': 6,
 'Value': 5,
 'atm': 16,
 'big': 18,
 'comm': 12,
 'correction_minus': 21,
 'correction_plus': 20,
 'dining': 9,
 'giving': 13,
 'groceries': 10,
 'indo': 17,
 'makan': 8,
 'other': 14,
 'recurr': 15,
 'saving': 19,
 'transport': 11}

In [28]:
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row_index = 38
for item in result:
    if item.username == 'feifei' and item.tanggal_datetime.day>4 and item.tanggal_datetime.day<12 and item.tanggal_datetime.month==10:
    #if item.username == 'billy' and item.tanggal_datetime.day>4 and item.tanggal_datetime.day<12 and item.tanggal_datetime.month==10:
    #if item.username == 'feifei' and ((item.tanggal_datetime.day>27 and item.tanggal_datetime.month==9) or ((item.tanggal_datetime.day<5 and item.tanggal_datetime.month==10))):
    #if item.username == 'billy' and ((item.tanggal_datetime.day>27 and item.tanggal_datetime.month==9) or ((item.tanggal_datetime.day<5 and item.tanggal_datetime.month==10))):
        if row_index == 38:
            for i in range (8,22): 
                new_work_sheet.cell(column=i,row=row_index).value = 0
            if item.Cash:
                new_work_sheet.cell(column=titles_column[item.Type],row=row_index).value = item.Value
        else:
            for i in range (8,22): 
                new_work_sheet.cell(column=i,row=row_index).value = new_work_sheet.cell(column=i,row=row_index-1).value
            if item.Cash:
                new_work_sheet.cell(column=titles_column[item.Type],row=row_index).value = item.Value+\
                  new_work_sheet.cell(column=titles_column[item.Type],row=row_index-1).value
        
        new_work_sheet.cell(column=titles_column['Keterangan'],row=row_index).value = item.Description
        new_work_sheet.cell(column=titles_column['CashYa?'],row=row_index).value = item.Cash
        new_work_sheet.cell(column=titles_column['Value'],row=row_index).value = item.Value
        new_work_sheet.cell(column=titles_column['Saldo'],row=row_index).value = item.Saldo
        new_work_sheet.cell(column=titles_column['Category'],row=row_index).value = item.Type
        
        new_work_sheet[get_column_letter(titles_column['Date'])+str(row_index)] = item.tanggal_datetime
        new_work_sheet[get_column_letter(titles_column['Date'])+str(row_index)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
        new_work_sheet[get_column_letter(titles_column['InputDate'])+str(row_index)] = item.createdAt_datetime
        new_work_sheet[get_column_letter(titles_column['InputDate'])+str(row_index)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
        row_index = row_index + 1


In [68]:
new_work_sheet.column_dimensions['A'].width = 32
new_work_sheet.column_dimensions['B'].width = 32
new_work_sheet.column_dimensions['C'].width = 25


In [24]:
for item in result:
    #if item.username == 'feifei' and item.tanggal_datetime.day>20 and item.tanggal_datetime.day<28 and item.tanggal_datetime.month==9:
    if item.username == 'billy' and item.tanggal_datetime.day>4 and item.tanggal_datetime.day<12 and item.tanggal_datetime.month==10:
    #if item.username == 'billy' and item.tanggal_datetime.day>4 and item.tanggal_datetime.month==10:
        print item.tanggal_datetime, item.createdAt_datetime, item.username, item.Description, item.Cash, item.Value, item.Saldo, item.Type

2015-10-05 12:30:40.533000+08:00 2015-10-05 12:31:03.256000+08:00 billy Lunch True 5 14 makan
2015-10-05 12:31:02.761000+08:00 2015-10-05 15:03:13.029000+08:00 billy Pagi 2 di pioneer True 100 114 atm
2015-10-05 19:43:14.238000+08:00 2015-10-05 19:43:34.744000+08:00 billy Dinner True 5 109 makan
2015-10-05 19:43:33.811000+08:00 2015-10-05 19:45:02.002000+08:00 billy Error True 1 110 correction_plus
2015-10-07 19:03:39.133000+08:00 2015-10-07 19:04:02.807000+08:00 billy Roti pagi True 2 108 makan
2015-10-07 19:03:59.938000+08:00 2015-10-07 19:05:28.786000+08:00 billy Lunch True 4 104 makan
2015-10-07 19:05:25.794000+08:00 2015-10-07 19:09:11.065000+08:00 billy Dinner True 5 99 makan
2015-10-08 15:11:03.150000+08:00 2015-10-08 15:11:23.074000+08:00 billy Eat True 11 88 makan
2015-10-09 19:09:40.033000+08:00 2015-10-09 19:09:52.715000+08:00 billy Breakfast and lunch True 12 76 makan
2015-10-09 19:09:51.158000+08:00 2015-10-09 20:34:16.318000+08:00 billy Dinner at mcd True 6 70 makan
2015-

In [25]:
titles = ['makan','dining','groceries','transport','comm','giving','other','recurr','atm','indo','big','saving','correction_plus','correction_minus']
column_index = 8
for string in titles:
    new_work_sheet.cell(column=column_index,row=20).value = string
    column_index = column_index + 1
titles_column = dict(zip(titles,range(8,22)))
titles_column

{'atm': 16,
 'big': 18,
 'comm': 12,
 'correction_minus': 21,
 'correction_plus': 20,
 'dining': 9,
 'giving': 13,
 'groceries': 10,
 'indo': 17,
 'makan': 8,
 'other': 14,
 'recurr': 15,
 'saving': 19,
 'transport': 11}

In [5]:
print result["results"][0]["createdAt"]

2015-09-14T12:10:00.793Z


In [19]:
connection = httplib.HTTPSConnection('api.parse.com', 443)
params = urllib.urlencode({"where":json.dumps({
       "createdAt": {
             "$gte": '2015-09-14T12:42:52.441Z'
       }
     })})
connection.connect()
connection.request('GET', '/1/classes/Expenses?%s' % params, '', {
       "X-Parse-Application-Id": "dCxojn2rtpic9RlfY6mjwTqWHNjHqsUOvwzPTl0Y",
       "X-Parse-REST-API-Key": "XzDkpbFjOPkJhW9fBHCPZGZ3mGgwk1C0GHFVeMKb"
     })
result = json.loads(connection.getresponse().read())
print result

{u'results': [{u'Description': u'dinner', u'objectId': u'9YC72WDkjy', u'Value': 3.84, u'Saldo': 0, u'Tanggal': {u'iso': u'2015-09-14T12:42:40.442Z', u'__type': u'Date'}, u'updatedAt': u'2015-09-14T12:42:52.441Z', u'Type': u'makan', u'createdAt': u'2015-09-14T12:42:52.441Z'}]}


In [20]:
len(result['results'])

1

In [1]:
import datetime
import pytz
import dateutil.parser

tz =  pytz.timezone('Asia/Singapore')

when = '2015-09-14T12:42:52.441Z'

_when = dateutil.parser.parse(when)

if not _when.tzinfo:
  _when = tz.localize(_when)
print _when

2015-09-14 12:42:52.441000+00:00


In [15]:
_when

datetime.datetime(2015, 9, 14, 12, 8, 15, 618000, tzinfo=tzutc())

In [16]:
tz

<DstTzInfo 'Asia/Singapore' LMT+6:55:00 STD>

In [22]:
print _when.astimezone(tz)

2015-09-14 20:42:52.441000+08:00


In [3]:
__when = _when.astimezone(tz)

In [4]:
__when

datetime.datetime(2015, 9, 14, 20, 42, 52, 441000, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>)

In [6]:
print __when > _when
print __when < _when
print __when == _when


False
False
True


In [11]:
__when.hour, __when.minute, __when.second, __when.microsecond

(20, 42, 52, 441000)

In [13]:
__when.day, __when.year, __when.month

(14, 2015, 9)

In [1]:
import sys,csv

In [2]:
f = open('TransactionHistory_20150920092712.csv', 'rt')
try:
    reader = csv.reader(f)
    for row in reader:
        print row
        if (len(row) == 2) and row[0] == 'Available ':
            _saldo_ocbc = float(row[1].replace(',',''))
            print 'got saldo ', _saldo_ocbc
finally:
    f.close()

['Account details for:', 'Savings Account 622-7-109128']
['Available ', '5,982.19']
got saldo  5982.19
['Balance', '5,982.19']
[]
['Transaction History ']
['Transaction date', 'Value date', 'Description', 'Withdrawals (SGD)', 'Deposits (SGD)']
['18/09/2015', '18/09/2015', 'PURCHASE        NETS', '30.00', '']
['', '', 'THANK YOU                CITY HALL  SMRT - CTH ']
['18/09/2015', '18/09/2015', 'PURCHASE        NETS', '1,010.00', '']
['', '', 'THANK YOU                ROBINSON  SINGPOST ']
['17/09/2015', '17/09/2015', 'FAST PAYMENT', '970.00', '']
['', '', 'Transferred from OCBC UOVB 3693749407 OTHR Uncle Jeff ']
['17/09/2015', '17/09/2015', '3RD PTY TRANSFER', '300.00', '']
['', '', 'VBS']
['17/09/2015', '17/09/2015', 'FAST PAYMENT', '1,000.00', '']
['', '', 'Transferred from OCBC DBSS 0204069022 OTHR Billy Saputra ']
['17/09/2015', '17/09/2015', 'PURCHASE        NETS', '20.25', '']
['', '', 'COMFORT TAXIS            THANK YOU  COMFORT TR ']
['16/09/2015', '16/09/2015', 'CASH WITHDRA

In [3]:
f = open('acb5e3dd9227f4b749d58dc810ed7059.P000000011915007.csv', 'rt')
try:
    reader = csv.reader(f)
    for row in reader:
        print row
        if (len(row) == 2) and row[0] == 'Available Balance:':
            _saldo_dbs = float(row[1].replace(',',''))
            print 'got saldo ', _saldo_dbs
finally:
    f.close()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Account Details For:', 'DBS Savings Plus Account 020-4-069022']
['Statement as at:', '20 Sep 2015 ']
[]
['Available Balance:', '1017.28']
got saldo  1017.28
['Ledger Balance:', '1070.78']
[]
['Transaction Date', 'Reference', 'Debit Amount', 'Credit Amount', 'Transaction Ref1', 'Transaction Ref2', 'Transaction Ref3']
[]
[]
['20 Sep 2015', 'POS', ' 89.00', ' ', 'COURTS', '', '', '']
['20 Sep 2015', 'POS', ' 19.35', ' ', 'NTUC FP', '', '', '']
['18 Sep 2015', 'D2P', ' 43.22', ' ', 'AXS           20150918205056730', '', '', '']
['18 Sep 2015', 'AWL', ' 100.00', ' ', '70252266', 'PIONEER MRT 2', '', '', '']
['17 Sep 2015', 'POS', ' 20.00', ' ', 'TRANSIT LI', '', '', '']
['17 Sep 2015', 'ICT', ' ', ' 1000.00', 'Transferred from OCBC', 'Z621611509170929251557150917092925', 'OTHR', '']
['13 Sep 2015', 'D2P', ' 15.00', ' ', 'AXS           20150913142305651', '', '', '']
['12 Sep 2015', 'AWL', ' 100.00', ' ', '70252266', 'NTUC CLEM MALL', '', '', '']
['12 Sep 2015

In [74]:
import datetime
import re

def valid_date(datestring):
        try:
                mat=re.match('(\d{2})[/.-](\d{2})[/.-](\d{4})$', datestring)
                if mat is not None:
                        datetime.datetime(*(map(int, mat.groups()[-1::-1])))
                        return True
        except: # ValueError:
                pass
        return False

In [75]:
import datetime
import re

def get_valid_date(datestring):
        try:
                mat=re.match('(\d{2})[/.-](\d{2})[/.-](\d{4})$', datestring)
                #print mat
                if mat is not None:
                        date = datetime.datetime(*(map(int, mat.groups()[-1::-1])))
                        print date
                        return date
        except: # ValueError:
                pass
        return None

In [76]:
def get_first(iterable, default=None):
    if iterable:
        for item in iterable:
            return item
    return default

In [77]:
f = open('TransactionHistory_20150920092712.csv', 'rt')
old_line = []
filtered_lines = []
g_first = True
try:
    reader = csv.reader(f)
    for line in reader:
        if valid_date(get_first(line)):
            #print old_line
            if (False == g_first):
                filtered_lines.append(old_line)
            else:
                g_first = False    
            old_line = line
        else:
            old_line= old_line + line
            
finally:
    f.close()
    for item in filtered_lines:
        print item

['18/09/2015', '18/09/2015', 'PURCHASE        NETS', '30.00', '', '', '', 'THANK YOU                CITY HALL  SMRT - CTH ']
['18/09/2015', '18/09/2015', 'PURCHASE        NETS', '1,010.00', '', '', '', 'THANK YOU                ROBINSON  SINGPOST ']
['17/09/2015', '17/09/2015', 'FAST PAYMENT', '970.00', '', '', '', 'Transferred from OCBC UOVB 3693749407 OTHR Uncle Jeff ']
['17/09/2015', '17/09/2015', '3RD PTY TRANSFER', '300.00', '', '', '', 'VBS']
['17/09/2015', '17/09/2015', 'FAST PAYMENT', '1,000.00', '', '', '', 'Transferred from OCBC DBSS 0204069022 OTHR Billy Saputra ']
['17/09/2015', '17/09/2015', 'PURCHASE        NETS', '20.25', '', '', '', 'COMFORT TAXIS            THANK YOU  COMFORT TR ']
['16/09/2015', '16/09/2015', 'CASH WITHDRAWAL  ATM', '100.00', '', '', '', 'OCBC-AMOY ST FOOD CENTRE SINGAPORE']
['15/09/2015', '15/09/2015', 'PURCHASE        NETS', '12.70', '', '', '', 'HAVE A NCIE DAY!         GUARDIAN  GUARDIAN H ']
['14/09/2015', '14/09/2015', 'GIRO - SALARY', '', '5,08

In [78]:
import datetime
import pytz

tz =  pytz.timezone('Asia/Singapore')

from datetime import datetime

for item in filtered_lines:
    #print get_first(item)
    #_when = get_valid_date(get_first(item))
    _when = datetime.strptime(get_first(item), "%d/%m/%Y")
    _when2 = datetime.strptime(item[1], "%d/%m/%Y")
    if not _when.tzinfo:
      _when = tz.localize(_when)
    if not _when2.tzinfo:
      _when2 = tz.localize(_when2)
    info_string = ''
    for info in item[2::]:
        try:
            _tval = float(info)
        except:
            info_string = info_string + ' ' + info
    print _when , _when2, _tval, info_string

2015-09-18 00:00:00+08:00 2015-09-18 00:00:00+08:00 30.0  PURCHASE        NETS    THANK YOU                CITY HALL  SMRT - CTH 
2015-09-18 00:00:00+08:00 2015-09-18 00:00:00+08:00 30.0  PURCHASE        NETS 1,010.00    THANK YOU                ROBINSON  SINGPOST 
2015-09-17 00:00:00+08:00 2015-09-17 00:00:00+08:00 970.0  FAST PAYMENT    Transferred from OCBC UOVB 3693749407 OTHR Uncle Jeff 
2015-09-17 00:00:00+08:00 2015-09-17 00:00:00+08:00 300.0  3RD PTY TRANSFER    VBS
2015-09-17 00:00:00+08:00 2015-09-17 00:00:00+08:00 300.0  FAST PAYMENT 1,000.00    Transferred from OCBC DBSS 0204069022 OTHR Billy Saputra 
2015-09-17 00:00:00+08:00 2015-09-17 00:00:00+08:00 20.25  PURCHASE        NETS    COMFORT TAXIS            THANK YOU  COMFORT TR 
2015-09-16 00:00:00+08:00 2015-09-16 00:00:00+08:00 100.0  CASH WITHDRAWAL  ATM    OCBC-AMOY ST FOOD CENTRE SINGAPORE
2015-09-15 00:00:00+08:00 2015-09-15 00:00:00+08:00 12.7  PURCHASE        NETS    HAVE A NCIE DAY!         GUARDIAN  GUARDIAN H 
2

In [6]:
mod_list = []
count = 1
f = open('TransactionHistory_20150920092712.csv', 'rt')
try:
    reader = csv.reader(f)
    for line in reader:
#for line in file(filename):
    # strip ending whitespaces including newline char
      #line = line.rstrip()
    # concatenate every two lines
      if count % 2 == 0:
        #mod_list.append(old_line+line)
        old_line= old_line + line
        print old_line
      else:
        old_line = line
      count += 1
    #print(mod_list)
finally:
    f.close()

['Account details for:', 'Savings Account 622-7-109128', 'Available ', '5,982.19']
['Balance', '5,982.19']
['Transaction History ', 'Transaction date', 'Value date', 'Description', 'Withdrawals (SGD)', 'Deposits (SGD)']
['18/09/2015', '18/09/2015', 'PURCHASE        NETS', '30.00', '', '', '', 'THANK YOU                CITY HALL  SMRT - CTH ']
['18/09/2015', '18/09/2015', 'PURCHASE        NETS', '1,010.00', '', '', '', 'THANK YOU                ROBINSON  SINGPOST ']
['17/09/2015', '17/09/2015', 'FAST PAYMENT', '970.00', '', '', '', 'Transferred from OCBC UOVB 3693749407 OTHR Uncle Jeff ']
['17/09/2015', '17/09/2015', '3RD PTY TRANSFER', '300.00', '', '', '', 'VBS']
['17/09/2015', '17/09/2015', 'FAST PAYMENT', '1,000.00', '', '', '', 'Transferred from OCBC DBSS 0204069022 OTHR Billy Saputra ']
['17/09/2015', '17/09/2015', 'PURCHASE        NETS', '20.25', '', '', '', 'COMFORT TAXIS            THANK YOU  COMFORT TR ']
['16/09/2015', '16/09/2015', 'CASH WITHDRAWAL  ATM', '100.00', '', '', '

In [79]:
import re

def valid_date_DBS(datestring):
        try:
                mat=re.match('^((([0-9])|([0-2][0-9])|([3][0-1]))\ (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\ \d{4})|(T\+[0-9]+)$', datestring)
                #mat=re.match('^[01][0-9] (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4}$', datestring)
                if mat is not None:
                        return True
        except: # ValueError:
                pass
        return False

In [80]:
f = open('acb5e3dd9227f4b749d58dc810ed7059.P000000011915007.csv', 'rt')
old_line = []
filtered_lines = []
try:
    reader = csv.reader(f)
    for line in reader:
        if valid_date_DBS(get_first(line)):
            #print old_line
            #old_line = line
            #print line
            filtered_lines.append(line)
        #else:
            #old_line= old_line + line
finally:
    f.close()
    for item in filtered_lines:
        print item

['20 Sep 2015', 'POS', ' 89.00', ' ', 'COURTS', '', '', '']
['20 Sep 2015', 'POS', ' 19.35', ' ', 'NTUC FP', '', '', '']
['18 Sep 2015', 'D2P', ' 43.22', ' ', 'AXS           20150918205056730', '', '', '']
['18 Sep 2015', 'AWL', ' 100.00', ' ', '70252266', 'PIONEER MRT 2', '', '', '']
['17 Sep 2015', 'POS', ' 20.00', ' ', 'TRANSIT LI', '', '', '']
['17 Sep 2015', 'ICT', ' ', ' 1000.00', 'Transferred from OCBC', 'Z621611509170929251557150917092925', 'OTHR', '']
['13 Sep 2015', 'D2P', ' 15.00', ' ', 'AXS           20150913142305651', '', '', '']
['12 Sep 2015', 'AWL', ' 100.00', ' ', '70252266', 'NTUC CLEM MALL', '', '', '']
['12 Sep 2015', 'POS', ' 10.95', ' ', 'NTUC FAIRP', '', '', '']
['10 Sep 2015', 'MST', ' 13.80', ' ', 'PAYPAL *MICROSOFTRE OF 40 29 07SEP', '5548-2740-7025-2266', '', '']
['09 Sep 2015', 'MST', ' 1.80', ' ', 'PAYPAL *DNREGISTRAR    40 29 07SEP', '5548-2740-7025-2266', '', '']
['06 Sep 2015', 'POS', ' 27.07', ' ', 'ETERNAL HA', '', '', '']
['04 Sep 2015', 'AWL', ' 100

In [81]:
import datetime
import pytz

tz =  pytz.timezone('Asia/Singapore')

from datetime import datetime


for item in filtered_lines:
    #print get_first(item).split()[0:3]
    day, month, year = get_first(item).split()[0:3]
    _when = datetime.strptime("{yyyy} {Mmm} {dd}".format(yyyy=year, Mmm=month, dd=day), "%Y %b %d")
    if not _when.tzinfo:
      _when = tz.localize(_when)
    info_string = ''
    for info in item[2::]:
        try:
            _tval = float(info)
        except:
            info_string = info_string + ' ' + info
    print _when, _tval, info_string

2015-09-20 00:00:00+08:00 89.0    COURTS   
2015-09-20 00:00:00+08:00 19.35    NTUC FP   
2015-09-18 00:00:00+08:00 43.22    AXS           20150918205056730   
2015-09-18 00:00:00+08:00 70252266.0    PIONEER MRT 2   
2015-09-17 00:00:00+08:00 20.0    TRANSIT LI   
2015-09-17 00:00:00+08:00 1000.0    Transferred from OCBC Z621611509170929251557150917092925 OTHR 
2015-09-13 00:00:00+08:00 15.0    AXS           20150913142305651   
2015-09-12 00:00:00+08:00 70252266.0    NTUC CLEM MALL   
2015-09-12 00:00:00+08:00 10.95    NTUC FAIRP   
2015-09-10 00:00:00+08:00 13.8    PAYPAL *MICROSOFTRE OF 40 29 07SEP 5548-2740-7025-2266  
2015-09-09 00:00:00+08:00 1.8    PAYPAL *DNREGISTRAR    40 29 07SEP 5548-2740-7025-2266  
2015-09-06 00:00:00+08:00 27.07    ETERNAL HA   
2015-09-04 00:00:00+08:00 70252266.0    PIONEER MRT 2   
2015-09-04 00:00:00+08:00 48.6    WWF-DONATION           SI NG 03SEP 5548-2740-7025-2266  
2015-09-04 00:00:00+08:00 9.11    PAYPAL *NETFLIX.COM    40 29 02SEP 5548-2740-702